<a href="https://colab.research.google.com/github/agrimwood/medphys_helper/blob/main/CBCT_audit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd
import numpy as np


In [ ]:
# Load K Factors (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4985946/)
k = {
    "Abdomen": 0.024,
    "Breast": 0.027,
    "Head": 0.002,
    "Head & Neck": 0.0057,
    "Thorax": 0.027,
    "Pelvis": 0.02
}

# Load UK Wide CBDI values (https://iopscience.iop.org/article/10.1088/1361-6560/ad88d1/pdf)
cbdi_truebeam = {
    "Clinical Site": ["Brain","Head & Neck","Breast","Lung 3D","Lung 4D","Prostate","Gynae"],
    "Median": [3.5,3.5,4.6,4.7,11.8,19.,19.],
    "IQR": [0.1,0.5,1.6,0.,0.9,3.8,3.6]
}
cbdi_all = {
    "Clinical Site": ["Brain","Head & Neck","Breast","Lung 3D","Lung 4D","Prostate","Gynae"],
    "Median": [2.6,3.5,4.1,4.7,11.5,19.,19.],
    "IQR": [2.2,2.9,2.4,1.3,1.9,6.2,5.9]
}

# Load OpenREM data
xl_files = glob.glob(f"/content/*.xlsx")
df = pd.concat([pd.read_excel(f, sheet_name="All data") for f in xl_files])
df_raw = df.copy()

In [ ]:
scan_stats = {}

# Earliest date
scan_stats["earliest_scan"] = df["Study date"].min()

# Latest date
scan_stats["latest_scan"] = df["Study date"].max()

# Number of sessions
scan_stats["n_scans"] = len(df)


In [ ]:
# Number of sessions after exclusions
df = df[(df["Test patient?"].isna()) & (df["Accession number"].notna()) & (df["Age"]>=15) & (df["Age"]<100) & (df["Sex"] != "O")]
scan_stats["n_scans_excl"] = len(df)

# Number of sessions by linac
scan_stats["n_scans_by_linac"] = df[df["No. events"]==1].groupby("Station name").size().to_dict()

# Number of sessions by protocol
scan_stats["n_scans_by_protocol"] = df[df["No. events"]==1].groupby("E1 Protocol").size().to_dict()

# Number of extended sessions by protocol
scan_stats["n_ext_scans_by_protocol"] = df[df["No. events"]>1].groupby("E1 Protocol").size().to_dict()


# Convert number of sessions to dataframe
n_sessions = {"E1 Protocol": list(scan_stats["n_scans_by_protocol"]), "Local Protocol": list(scan_stats["n_scans_by_protocol"]), "Number of sessions": [i for i in scan_stats["n_scans_by_protocol"].values()]}
n_sessions["E1 Protocol"].extend(list(scan_stats["n_ext_scans_by_protocol"]))
n_sessions["Local Protocol"].extend([i+" Extended" for i in scan_stats["n_ext_scans_by_protocol"]])
n_sessions["Number of sessions"].extend([i for i in scan_stats["n_ext_scans_by_protocol"].values()])
n_sessions = pd.DataFrame(n_sessions)
n_sessions

In [ ]:
# Phantom Size by protocol
scan_stats["phantom_sz_by_protocol"] = df.groupby("E1 Protocol")["E1 Phantom"].unique()[:].to_dict()
for i in scan_stats["phantom_sz_by_protocol"].keys():
  scan_stats["phantom_sz_by_protocol"][i]=float(scan_stats["phantom_sz_by_protocol"][i][0].replace(" cm",""))

# Convert to dataframe
phantom_size = pd.DataFrame({"E1 Protocol": scan_stats["phantom_sz_by_protocol"].keys(), "Phantom Size (cm)": scan_stats["phantom_sz_by_protocol"].values()})
phantom_size

In [ ]:
# CTDI Median by protocol (non extended)
scan_stats["ctdi_median_by_protocol"] = df[df["No. events"]==1].groupby("E1 Protocol")["E1 CTDIvol"].median().to_dict()

# CTDI IQR by protocol (non extended)
scan_stats["ctdi_IQR_by_protocol"] = {}
for i in df[df["No. events"]==1]["E1 Protocol"].unique():
  scan_stats["ctdi_IQR_by_protocol"][i] = df[df["E1 Protocol"]==i]["E1 CTDIvol"].quantile(0.75) - df[df["E1 Protocol"]==i]["E1 CTDIvol"].quantile(0.25)


# CTDI Median by extended protocol
scan_stats["ext_ctdi_median_by_protocol"] = df[df["No. events"]==2].groupby("E1 Protocol")["E1 CTDIvol"].median().to_dict()

# CTDI IQR by extended protocol
scan_stats["ext_ctdi_IQR_by_protocol"] = {}
for i in df[df["No. events"]==2]["E1 Protocol"].unique():
  scan_stats["ext_ctdi_IQR_by_protocol"][i] = df[df["E1 Protocol"]==i]["E1 CTDIvol"].quantile(0.75) - df[df["E1 Protocol"]==i]["E1 CTDIvol"].quantile(0.25)


# convert back to dataframe
ctdi_local = {"E1 Protocol": [], "Local Protocol": [], "Local CTDI Median": [], "Local CTDI IQR": []}
for i in scan_stats["ctdi_median_by_protocol"].keys():
  ctdi_local["Local Protocol"].append(i)
  ctdi_local["E1 Protocol"].append(i)
  ctdi_local["Local CTDI Median"].append(scan_stats["ctdi_median_by_protocol"][i])
  ctdi_local["Local CTDI IQR"].append(scan_stats["ctdi_IQR_by_protocol"][i])
for i in scan_stats["ext_ctdi_median_by_protocol"].keys():
  ctdi_local["Local Protocol"].append(i+" Extended")
  ctdi_local["E1 Protocol"].append(i)
  ctdi_local["Local CTDI Median"].append(scan_stats["ext_ctdi_median_by_protocol"][i])
  ctdi_local["Local CTDI IQR"].append(scan_stats["ext_ctdi_IQR_by_protocol"][i])

ctdi_local = pd.DataFrame(ctdi_local)
ctdi_local

In [ ]:
# Total scan length median by protocol
df.astype({'E1 Scanning length': 'float', 'E2 Scanning length': 'float'}).dtypes
df.loc[(df["E2 Scanning length"].isna()), ("E2 Scanning length")] = 0
df["Total Scan Length"] = df["E1 Scanning length"]/10 + df["E2 Scanning length"]/10
scan_stats["scan_length_by_protocol"] = df[df["No. events"]==1].groupby("E1 Protocol")["Total Scan Length"].median().to_dict()

# Total scan length median by extended protocol
scan_stats["ext_scan_length_by_protocol"] = df[df["No. events"]==2].groupby("E1 Protocol")["Total Scan Length"].median().to_dict()


# convert back to dataframe
scan_length = {"E1 Protocol": [i for i in scan_stats["scan_length_by_protocol"]], "Local Protocol": [i for i in scan_stats["scan_length_by_protocol"]], "Scan Length Median": [i for i in scan_stats["scan_length_by_protocol"].values()]}
scan_length['E1 Protocol'].extend([i for i in scan_stats["ext_scan_length_by_protocol"]])
scan_length["Local Protocol"].extend([i+" Extended" for i in scan_stats["ext_scan_length_by_protocol"]])
scan_length["Scan Length Median"].extend([i for i in scan_stats["ext_scan_length_by_protocol"].values()])
scan_length = pd.DataFrame(scan_length)
scan_length


In [ ]:
# Total DLP median by protocol
scan_stats["dlp_median_by_protocol"] = df[df['No. events']==1].groupby("E1 Protocol")["DLP total (mGy.cm)"].median().to_dict()

# Total DLP IQR by protocol
scan_stats["dlp_IQR_by_protocol"] = {}
for i in df[df["No. events"]==1]["E1 Protocol"].unique():
  scan_stats["dlp_IQR_by_protocol"][i] = df[df["E1 Protocol"]==i]["DLP total (mGy.cm)"].quantile(0.75) - df[df["E1 Protocol"]==i]["DLP total (mGy.cm)"].quantile(0.25)


# Total DLP median by extended protocol
scan_stats["ext_dlp_median_by_protocol"] = df[df['No. events']==2].groupby("E1 Protocol")["DLP total (mGy.cm)"].median().to_dict()

# Total DLP IQR by extended protocol
scan_stats["ext_dlp_IQR_by_protocol"] = {}
for i in df[df["No. events"]==2]["E1 Protocol"].unique():
  scan_stats["ext_dlp_IQR_by_protocol"][i] = df[df["E1 Protocol"]==i]["DLP total (mGy.cm)"].quantile(0.75) - df[df["E1 Protocol"]==i]["DLP total (mGy.cm)"].quantile(0.25)


# convert back to dataframe
dlp_local = {"E1 Protocol": [], "Local Protocol": [], "Local DLP Median": [], "Local DLP IQR": []}
for i in scan_stats["dlp_median_by_protocol"].keys():
  dlp_local["Local Protocol"].append(i)
  dlp_local["E1 Protocol"].append(i)
  dlp_local["Local DLP Median"].append(scan_stats["dlp_median_by_protocol"][i])
  dlp_local["Local DLP IQR"].append(scan_stats["dlp_IQR_by_protocol"][i])
for i in scan_stats["ext_dlp_median_by_protocol"].keys():
  dlp_local["Local Protocol"].append(i+" Extended")
  dlp_local["E1 Protocol"].append(i)
  dlp_local["Local DLP Median"].append(scan_stats["ext_dlp_median_by_protocol"][i])
  dlp_local["Local DLP IQR"].append(scan_stats["ext_dlp_IQR_by_protocol"][i])
dlp_local = pd.DataFrame(dlp_local)
dlp_local


In [ ]:
# Effective dose from local DLP
'''
Manually cross-reference K Factors to the appropriate Local Protocol to calculate effective dose.
For Head_POST calculate dose for both Brain and Head & Neck

Effective Dose = DLP * k

'''

d = {"E1 Protocol": [], "k Site": [], "k Factor": []}
for i in scan_stats["dlp_median_by_protocol"].keys():
  if "abdomen" in i.lower():
    d["E1 Protocol"].append(i)
    d["k Site"].append("Abdomen")
    d["k Factor"].append(k["Abdomen"])
  elif "head" in i.lower():
    d["E1 Protocol"].append(i)
    d["k Site"].append("Head")
    d["k Factor"].append(k["Head"])
    d["E1 Protocol"].append(i)
    d["k Site"].append("Head & Neck")
    d["k Factor"].append(k["Head & Neck"])
  elif "breast" in i.lower():
    d["E1 Protocol"].append(i)
    d["k Site"].append("Breast")
    d["k Factor"].append(k["Breast"])
  elif "thorax" in i.lower():
    d["E1 Protocol"].append(i)
    d["k Site"].append("Thorax")
    d["k Factor"].append(k["Thorax"])
  elif "pelvis" in i.lower():
    d["E1 Protocol"].append(i)
    d["k Site"].append("Pelvis")
    d["k Factor"].append(k["Pelvis"])

ed_local = pd.DataFrame(d)
ed_local = ed_local.merge(dlp_local, on="E1 Protocol")
ed_local["Local Effective Dose Median"] = ed_local["Local DLP Median"]*ed_local["k Factor"]
ed_local["Local Effective Dose IQR"] = ed_local["Local DLP IQR"]*ed_local["k Factor"]
ed_local

In [ ]:
# Truebeam CTDI from Woods 2024
'''
Assign relevant UK-wide Truebeam CTDIs to Local Protocols by
manually cross-referencing the appropriate Clinical Sites from Woods 2024 to Local Protocols.
'''
truebeam_ctdi = {"E1 Protocol": [], "Local Protocol": [], "Clinical Site": [], "TB CTDI Median": [], "TB CTDI IQR": []}
for i,p in zip(ctdi_local["Local Protocol"],ctdi_local["E1 Protocol"]):
  if "head" in i.lower():
    j = cbdi_truebeam["Clinical Site"].index("Brain")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)
    j = cbdi_truebeam["Clinical Site"].index("Head & Neck")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)
  elif "abdomen" in i.lower():
    j = cbdi_truebeam["Clinical Site"].index("Prostate")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)
  elif "breast" in i.lower():
    j = cbdi_truebeam["Clinical Site"].index("Breast")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)
  elif "4d thorax" in i.lower():
    j = cbdi_truebeam["Clinical Site"].index("Lung 4D")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)
  elif "thorax" in i.lower():
    j = cbdi_truebeam["Clinical Site"].index("Lung 3D")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)
  elif "pelvis" in i.lower():
    j = cbdi_truebeam["Clinical Site"].index("Prostate")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)
    j = cbdi_truebeam["Clinical Site"].index("Gynae")
    truebeam_ctdi["TB CTDI Median"].append(cbdi_truebeam["Median"][j])
    truebeam_ctdi["TB CTDI IQR"].append(cbdi_truebeam["IQR"][j])
    truebeam_ctdi["Local Protocol"].append(i)
    truebeam_ctdi["Clinical Site"].append(cbdi_truebeam["Clinical Site"][j])
    truebeam_ctdi["E1 Protocol"].append(p)

truebeam_ctdi = pd.DataFrame(truebeam_ctdi)
truebeam_ctdi

In [ ]:
# All CTDI from Woods 2024
'''
Assign relevant UK-wide CTDIs to Local Protocols by
manually cross-referencing the appropriate Clinical Sites from Woods 2024 to Local Protocols.
'''

all_ctdi = {"E1 Protocol": [], "Local Protocol": [], "Clinical Site": [], "All CTDI Median": [], "All CTDI IQR": []}
for i,p in zip(ctdi_local["Local Protocol"],ctdi_local["E1 Protocol"]):
  if "head" in i.lower():
    j = cbdi_all["Clinical Site"].index("Brain")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])
    j = cbdi_all["Clinical Site"].index("Head & Neck")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])
  elif "abdomen" in i.lower():
    j = cbdi_all["Clinical Site"].index("Prostate")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])
  elif "breast" in i.lower():
    j = cbdi_all["Clinical Site"].index("Breast")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])
  elif "4d thorax" in i.lower():
    j = cbdi_all["Clinical Site"].index("Lung 4D")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])
  elif "thorax" in i.lower():
    j = cbdi_all["Clinical Site"].index("Lung 3D")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])
  elif "pelvis" in i.lower():
    j = cbdi_all["Clinical Site"].index("Prostate")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])
    j = cbdi_all["Clinical Site"].index("Gynae")
    all_ctdi["All CTDI Median"].append(cbdi_all["Median"][j])
    all_ctdi["All CTDI IQR"].append(cbdi_all["IQR"][j])
    all_ctdi["Local Protocol"].append(i)
    all_ctdi["E1 Protocol"].append(p)
    all_ctdi["Clinical Site"].append(cbdi_all["Clinical Site"][j])

all_ctdi = pd.DataFrame(all_ctdi)
all_ctdi

In [ ]:
# Truebeam DLP estimates from Woods 2024 CTDI
'''
Perform a rough estimate of DLP using Local Scan Lengths and UK-Wide Truebeam CTDI values from Woods 2024.

DLP = CTDI * Scan Length

'''
truebeam_dlp = truebeam_ctdi.copy()
truebeam_dlp = truebeam_dlp.merge(scan_length[["Local Protocol", "Scan Length Median"]], on="Local Protocol")
truebeam_dlp["Truebeam DLP Median"] = truebeam_dlp["TB CTDI Median"]*truebeam_dlp["Scan Length Median"]
truebeam_dlp["Truebeam DLP IQR"] = truebeam_dlp["TB CTDI IQR"]*truebeam_dlp["Scan Length Median"]
truebeam_dlp

In [ ]:
# All DLP estimates from Woods 2024 CTDI
'''
Perform a rough estimate of DLP using Local Scan Lengths and UK-Wide Truebeam CTDI values from Woods 2024.

DLP = CTDI * Scan Length

'''
all_dlp = all_ctdi.copy()
all_dlp = all_dlp.merge(scan_length[["Local Protocol", "Scan Length Median"]], on="Local Protocol")
all_dlp["All DLP Median"] = all_dlp["All CTDI Median"]*all_dlp["Scan Length Median"]
all_dlp["All DLP IQR"] = all_dlp["All CTDI IQR"]*all_dlp["Scan Length Median"]
all_dlp

In [ ]:
# Truebeam Effective Dose estimates for Woods 2024
truebeam_ed = truebeam_dlp.copy()
truebeam_ed["K Name"] = truebeam_ed["Local Protocol"]
truebeam_ed.loc[truebeam_ed["Clinical Site"]=="Brain","K Name"] =  truebeam_ed["Local Protocol"] + " " + "Head"
truebeam_ed.loc[truebeam_ed["Clinical Site"]=="Head & Neck","K Name"] =  truebeam_ed["Local Protocol"] + " " + "Head & Neck"

k_data = ed_local[["Local Protocol", "k Site", "k Factor"]].copy()
k_data["K Name"] = k_data["Local Protocol"]
k_data.loc[k_data["k Site"]=="Head","K Name"] =  k_data["Local Protocol"] + " " + "Head"
k_data.loc[k_data["k Site"]=="Head & Neck","K Name"] =  k_data["Local Protocol"] + " " + "Head & Neck"
truebeam_ed = truebeam_ed.merge(k_data[["K Name", "k Factor"]], on="K Name")
truebeam_ed["TB Edose Median"] = truebeam_ed["Truebeam DLP Median"]*truebeam_ed["k Factor"]
truebeam_ed["TB Edose IQR"] = truebeam_ed["Truebeam DLP IQR"]*truebeam_ed["k Factor"]
truebeam_ed

In [ ]:
# All Effective Dose estimates for Woods 2024
all_ed = all_dlp.copy()
all_ed["K Name"] = all_ed["Local Protocol"]
all_ed.loc[all_ed["Clinical Site"]=="Brain","K Name"] =  all_ed["Local Protocol"] + " " + "Head"
all_ed.loc[all_ed["Clinical Site"]=="Head & Neck","K Name"] =  all_ed["Local Protocol"] + " " + "Head & Neck"

k_data = ed_local[["Local Protocol", "k Site", "k Factor"]].copy()
k_data["K Name"] = k_data["Local Protocol"]
k_data.loc[k_data["k Site"]=="Head","K Name"] =  k_data["Local Protocol"] + " " + "Head"
k_data.loc[k_data["k Site"]=="Head & Neck","K Name"] =  k_data["Local Protocol"] + " " + "Head & Neck"
all_ed = all_ed.merge(k_data[["K Name", "k Factor"]], on="K Name")
all_ed["All Edose Median"] = all_ed["All DLP Median"]*all_ed["k Factor"]
all_ed["All Edose IQR"] = all_ed["All DLP IQR"]*all_ed["k Factor"]
all_ed

In [ ]:
# Make summary tables and export to excel workbook

# DRL Summary
drl_summary = phantom_size.merge(ctdi_local, on="E1 Protocol")
drl_summary = drl_summary.merge(ed_local[['Local Protocol','Local DLP Median','k Factor','Local Effective Dose Median']], on="Local Protocol")

# CTDI summary
ctdi_summary = ctdi_local.merge(truebeam_ctdi, on=["Local Protocol", "E1 Protocol"])
ctdi_summary = ctdi_summary.merge(all_ctdi, on=["Local Protocol","E1 Protocol","Clinical Site"])

# DLP summary
dlp_summary = dlp_local.merge(truebeam_dlp.drop(["Scan Length Median","TB CTDI Median","TB CTDI IQR"], axis=1), on=["Local Protocol", "E1 Protocol"])
dlp_summary = dlp_summary.merge(all_dlp.drop(["Scan Length Median","All CTDI Median","All CTDI IQR"], axis=1), on=["Local Protocol","Clinical Site","E1 Protocol"])

# Effective Dose summary
tb_ed = truebeam_ed[["E1 Protocol", "Local Protocol", "Clinical Site", "TB Edose Median", "TB Edose IQR"]].copy()
tb_ed["K Name"] = tb_ed["Local Protocol"]
tb_ed.loc[tb_ed["Clinical Site"]=="Brain","K Name"] =  tb_ed["Local Protocol"] + " " + "Head"
tb_ed.loc[tb_ed["Clinical Site"]=="Head & Neck","K Name"] =  tb_ed["Local Protocol"] + " " + "Head & Neck"
lc_ed = ed_local.copy()
lc_ed["K Name"] = lc_ed["Local Protocol"]
lc_ed.loc[lc_ed["k Site"]=="Head","K Name"] =  lc_ed["Local Protocol"] + " " + "Head"
lc_ed.loc[lc_ed["k Site"]=="Head & Neck","K Name"] =  lc_ed["Local Protocol"] + " " + "Head & Neck"
ed_summary = lc_ed[["E1 Protocol", "Local Protocol", "K Name", "Local Effective Dose Median", "Local Effective Dose IQR"]].merge(tb_ed, on=["Local Protocol", "K Name", "E1 Protocol"])
ed_summary = ed_summary.drop("K Name", axis=1).merge(all_ed[["E1 Protocol", "Local Protocol", "Clinical Site", "All Edose Median", "All Edose IQR"]], on=["Local Protocol","Clinical Site","E1 Protocol"])
ed_summary

# Export
summaries = {"DRL": drl_summary, "CTDI": ctdi_summary, "DLP": dlp_summary, "Dose": ed_summary, "Session Count": n_sessions, "Scan Lengths": scan_length, "Raw Data": df_raw, "Filtered Data": df}
with pd.ExcelWriter('CBCT_DRL_Summaries.xlsx') as writer:
  for i,df in summaries.items():
    df.to_excel(writer, index=False, sheet_name=i)

In [ ]:
print([scan_stats['earliest_scan'],scan_stats['latest_scan'], scan_stats['n_scans'], scan_stats['n_scans_excl']])
print([scan_stats['n_scans_by_linac']])
print([scan_stats['n_scans_by_protocol']])